In [51]:
from data_preparation import *
from get_real_data import *
from preprocessing import *

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import LSTM, Dense, Masking, Conv1D, Conv2D, Dropout, Flatten, MaxPooling1D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
folder_list = extract_file_list()

# Model

In [3]:
def initialize_model_CNN(input_shape):
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=input_shape))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3)))
    
    model.add(Flatten())
    model.add(Dense(units=200, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3)))
    model.add(Dense(units=100, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3)))
    model.add(Dense(units=16, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3)))
    model.add(Dense(units=16, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3)))
    model.add(Dense(units=3, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [4]:
patterns_to_test = ['rising_wedge', 'falling_wedge', 'double_top', 'double_bottom']
results_dict = {}

# Rising Wedge

In [5]:
X_real, y_real = real_data_pattern(patterns_to_test[0], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[0], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[0], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [6]:
y_all = np.array(y_joined)
X_train_rw, X_test_rw, y_train_rw, y_test_rw = train_test_split(X_pad, y_all, test_size=0.30)

X_train_rw = tf.convert_to_tensor(X_train_rw, np.float32)
y_train_rw = tf.convert_to_tensor(y_train_rw, np.int16)

2023-09-04 15:28:16.582066: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-04 15:28:16.582564: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_rw.shape[1:]
model_rw = initialize_model_CNN(input_shape)

model_rw.fit(
    X_train_rw,
    y_train_rw,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
21/21 [==============================] - 2s 36ms/step - loss: 449527.1250 - mae: 123.4331 - val_loss: 11219.5557 - val_mae: 58.6484
Epoch 2/100
21/21 [==============================] - 1s 25ms/step - loss: 65833.4062 - mae: 82.0896 - val_loss: 8848.4854 - val_mae: 54.5698
Epoch 3/100
21/21 [==============================] - 1s 26ms/step - loss: 25571.4922 - mae: 71.6717 - val_loss: 8389.1514 - val_mae: 57.2209
Epoch 4/100
21/21 [==============================] - 1s 25ms/step - loss: 14986.6406 - mae: 61.9748 - val_loss: 7525.7002 - val_mae: 49.3243
Epoch 5/100
21/21 [==============================] - 1s 28ms/step - loss: 9743.1641 - mae: 55.3179 - val_loss: 6750.5098 - val_mae: 48.4467
Epoch 6/100
21/21 [==============================] - 1s 28ms/step - loss: 11296.6553 - mae: 55.2379 - val_loss: 6461.5972 - val_mae: 47.4963
Epoch 7/100
21/21 [==============================] - 1s 25ms/step - loss: 8554.9531 - mae: 52.1845 - val_loss: 5804.7910 - val_mae: 43.4016
Epoch 8/100


In [8]:
X_test_rw = tf.convert_to_tensor(X_test_rw, np.float32)
y_test_rw = tf.convert_to_tensor(y_test_rw, np.int16)
res_rw = model_rw.evaluate(X_test_rw, y_test_rw)

results_dict['Rising Wedge'] = res_rw[1]

11/11 [==============================] - 0s 10ms/step - loss: 1983.1173 - mae: 22.0387


In [44]:
model_rw.save_weights('first_model_rw_weights.h5')

# Falling wedge

In [9]:
X_real, y_real = real_data_pattern(patterns_to_test[1], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[1], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[1], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [10]:
y_all = np.array(y_joined)
X_train_fw, X_test_fw, y_train_fw, y_test_fw = train_test_split(X_pad, y_all, test_size=0.30)

X_train_fw = tf.convert_to_tensor(X_train_fw, np.float32)
y_train_fw = tf.convert_to_tensor(y_train_fw, np.int16)

In [11]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_fw.shape[1:]
model_fw = initialize_model_CNN(input_shape)

model_fw.fit(
    X_train_fw,
    y_train_fw,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
14/14 [==============================] - 1s 24ms/step - loss: 76487.9922 - mae: 62.6327 - val_loss: 6684.9336 - val_mae: 35.7959
Epoch 2/100
14/14 [==============================] - 0s 14ms/step - loss: 6679.8223 - mae: 36.3095 - val_loss: 4242.2666 - val_mae: 29.6563
Epoch 3/100
14/14 [==============================] - 0s 15ms/step - loss: 3851.1912 - mae: 29.7000 - val_loss: 2896.6846 - val_mae: 28.7830
Epoch 4/100
14/14 [==============================] - 0s 14ms/step - loss: 2274.0491 - mae: 24.2230 - val_loss: 2277.5608 - val_mae: 24.8345
Epoch 5/100
14/14 [==============================] - 0s 14ms/step - loss: 1854.6697 - mae: 22.2338 - val_loss: 1554.9401 - val_mae: 21.9805
Epoch 6/100
14/14 [==============================] - 0s 14ms/step - loss: 1296.9087 - mae: 17.9701 - val_loss: 1449.8611 - val_mae: 21.2432
Epoch 7/100
14/14 [==============================] - 0s 14ms/step - loss: 1143.2477 - mae: 16.6056 - val_loss: 1136.1622 - val_mae: 19.7679
Epoch 8/100
14/14 [

In [12]:
X_test_fw = tf.convert_to_tensor(X_test_fw, np.float32)
y_test_fw = tf.convert_to_tensor(y_test_fw, np.int16)
res_fw = model_fw.evaluate(X_test_fw, y_test_fw)

results_dict['Falling Wedge'] = res_fw[1]

8/8 [==============================] - 0s 6ms/step - loss: 248.9709 - mae: 6.9889


# Double Top

In [13]:
X_real, y_real = real_data_pattern(patterns_to_test[2], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[2], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[2], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [14]:
y_all = np.array(y_joined)
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X_pad, y_all, test_size=0.30)

X_train_dt = tf.convert_to_tensor(X_train_dt, np.float32)
y_train_dt = tf.convert_to_tensor(y_train_dt, np.int16)

In [15]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_dt.shape[1:]
model_dt = initialize_model_CNN(input_shape)

model_dt.fit(
    X_train_dt,
    y_train_dt,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
14/14 [==============================] - 1s 26ms/step - loss: 76743.5703 - mae: 72.5666 - val_loss: 11534.0459 - val_mae: 49.8026
Epoch 2/100
14/14 [==============================] - 0s 16ms/step - loss: 102098.8438 - mae: 67.2756 - val_loss: 9900.8633 - val_mae: 47.4017
Epoch 3/100
14/14 [==============================] - 0s 18ms/step - loss: 934814.1250 - mae: 85.5220 - val_loss: 10677.6924 - val_mae: 45.7824
Epoch 4/100
14/14 [==============================] - 0s 17ms/step - loss: 51810.4336 - mae: 54.7192 - val_loss: 10193.3984 - val_mae: 43.2995
Epoch 5/100
14/14 [==============================] - 0s 18ms/step - loss: 34070.3164 - mae: 49.8616 - val_loss: 9583.3184 - val_mae: 46.2722
Epoch 6/100
14/14 [==============================] - 0s 16ms/step - loss: 12381.3320 - mae: 45.5336 - val_loss: 8692.3311 - val_mae: 44.5067
Epoch 7/100
14/14 [==============================] - 0s 16ms/step - loss: 8806.5322 - mae: 41.8756 - val_loss: 7798.0542 - val_mae: 40.7303
Epoch 8/1

In [17]:
X_test_dt = tf.convert_to_tensor(X_test_dt, np.float32)
y_test_dt = tf.convert_to_tensor(y_test_dt, np.int16)
res_dt = model_dt.evaluate(X_test_dt, y_test_dt)

results_dict['Double Top'] = res_dt[1]

8/8 [==============================] - 0s 5ms/step - loss: 2598.5349 - mae: 25.8533


# Double Bottom

In [18]:
X_real, y_real = real_data_pattern(patterns_to_test[3], model_type='full')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[3], model_type='full')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[3], model_type='full')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [19]:
y_all = np.array(y_joined)
X_train_db, X_test_db, y_train_db, y_test_db = train_test_split(X_pad, y_all, test_size=0.30)

X_train_db = tf.convert_to_tensor(X_train_db, np.float32)
y_train_db = tf.convert_to_tensor(y_train_db, np.int16)

In [20]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_db.shape[1:]
model_db = initialize_model_CNN(input_shape)

model_db.fit(
    X_train_db,
    y_train_db,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
13/13 [==============================] - 1s 24ms/step - loss: 14937.3643 - mae: 50.0123 - val_loss: 22834.8359 - val_mae: 66.7170
Epoch 2/100
13/13 [==============================] - 0s 15ms/step - loss: 40038.9297 - mae: 56.1952 - val_loss: 10914.7725 - val_mae: 55.2011
Epoch 3/100
13/13 [==============================] - 0s 14ms/step - loss: 26670.0527 - mae: 48.7399 - val_loss: 10233.1465 - val_mae: 50.6159
Epoch 4/100
13/13 [==============================] - 0s 14ms/step - loss: 6780.0850 - mae: 38.6671 - val_loss: 8626.2236 - val_mae: 49.0487
Epoch 5/100
13/13 [==============================] - 0s 14ms/step - loss: 6692.6416 - mae: 38.9577 - val_loss: 8818.4893 - val_mae: 49.4043
Epoch 6/100
13/13 [==============================] - 0s 14ms/step - loss: 5528.7051 - mae: 36.5205 - val_loss: 8162.5410 - val_mae: 50.2894
Epoch 7/100
13/13 [==============================] - 0s 14ms/step - loss: 5196.0825 - mae: 37.9164 - val_loss: 7479.1025 - val_mae: 46.7380
Epoch 8/100
13

In [21]:
X_test_db = tf.convert_to_tensor(X_test_db, np.float32)
y_test_db = tf.convert_to_tensor(y_test_db, np.int16)
res_db = model_db.evaluate(X_test_db, y_test_db)

results_dict['Double Bottom'] = res_db[1]

7/7 [==============================] - 0s 5ms/step - loss: 2078.9038 - mae: 16.5239


# Results

In [22]:
results_df = pd.DataFrame(results_dict, index=[1])
results_df

,Rising Wedge,Falling Wedge,Double Top,Double Bottom
1,22.038683,6.988884,25.853342,16.52389


# Pattern Present or not?

In [23]:
def initialize_model_CNN(input_shape):
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=input_shape))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))),
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, activation='relu', kernel_size=3))
    
    model.add(Flatten())
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Rising Wedge

In [24]:
accuracy_dict = {}

In [25]:
X_real, y_real = real_data_pattern(patterns_to_test[0], model_type='categorise')

X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[0], model_type='categorise')

X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[0], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [26]:
y_all = np.array(y_joined)

X_train_rwp, X_test_rwp, y_train_rwp, y_test_rwp = train_test_split(X_pad, y_all, test_size=0.30)

X_train_rwp = tf.convert_to_tensor(X_train_rwp, np.float32)
y_train_rwp = tf.convert_to_tensor(y_train_rwp, np.int16)

In [27]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_rwp.shape[1:]
model_rwp = initialize_model_CNN(input_shape)

model_rwp.fit(
    X_train_rwp,
    y_train_rwp,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
21/21 [==============================] - 1s 18ms/step - loss: 7.8128 - accuracy: 0.5809 - val_loss: 0.9231 - val_accuracy: 0.7669
Epoch 2/100
21/21 [==============================] - 0s 12ms/step - loss: 2.3079 - accuracy: 0.7180 - val_loss: 0.2809 - val_accuracy: 0.9080
Epoch 3/100
21/21 [==============================] - 0s 12ms/step - loss: 0.3273 - accuracy: 0.8706 - val_loss: 0.1885 - val_accuracy: 0.9632
Epoch 4/100
21/21 [==============================] - 0s 11ms/step - loss: 1.9606 - accuracy: 0.9014 - val_loss: 0.6020 - val_accuracy: 0.8098
Epoch 5/100
21/21 [==============================] - 0s 11ms/step - loss: 0.3063 - accuracy: 0.9199 - val_loss: 0.1464 - val_accuracy: 0.9693
Epoch 6/100
21/21 [==============================] - 0s 11ms/step - loss: 0.1978 - accuracy: 0.9476 - val_loss: 0.1432 - val_accuracy: 0.9693
Epoch 7/100
21/21 [==============================] - 0s 12ms/step - loss: 0.1490 - accuracy: 0.9661 - val_loss: 0.1375 - val_accuracy: 0.9755
Epoch 

In [28]:
X_test_rwp = tf.convert_to_tensor(X_test_rwp, np.float32)
y_test_rwp = tf.convert_to_tensor(y_test_rwp, np.int16)
res_rwp = model_rwp.evaluate(X_test_rwp, y_test_rwp)

accuracy_dict['Rising Wedge_class'] = res_rwp[1]

11/11 [==============================] - 0s 5ms/step - loss: 0.4529 - accuracy: 0.9741


# Falling Wedge

In [29]:
X_real, y_real = real_data_pattern(patterns_to_test[1], model_type='categorise')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[1], model_type='categorise')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[1], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [30]:
y_all = np.array(y_joined)

X_train_fwp, X_test_fwp, y_train_fwp, y_test_fwp = train_test_split(X_pad, y_all, test_size=0.30)

X_train_fwp = tf.convert_to_tensor(X_train_fwp, np.float32)
y_train_fwp = tf.convert_to_tensor(y_train_fwp, np.int16)

In [31]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_fwp.shape[1:]
model_fwp = initialize_model_CNN(input_shape)

model_fwp.fit(
    X_train_fwp,
    y_train_fwp,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
14/14 [==============================] - 1s 22ms/step - loss: 50.3083 - accuracy: 0.5877 - val_loss: 15.0339 - val_accuracy: 0.6545
Epoch 2/100
14/14 [==============================] - 0s 14ms/step - loss: 6.8213 - accuracy: 0.8360 - val_loss: 5.0435 - val_accuracy: 0.8909
Epoch 3/100
14/14 [==============================] - 0s 13ms/step - loss: 0.2289 - accuracy: 0.9590 - val_loss: 0.3820 - val_accuracy: 0.7727
Epoch 4/100
14/14 [==============================] - 0s 12ms/step - loss: 0.1957 - accuracy: 0.9362 - val_loss: 0.1114 - val_accuracy: 0.9909
Epoch 5/100
14/14 [==============================] - 0s 14ms/step - loss: 0.1577 - accuracy: 0.9658 - val_loss: 0.1277 - val_accuracy: 0.9545
Epoch 6/100
14/14 [==============================] - 0s 12ms/step - loss: 0.1150 - accuracy: 0.9886 - val_loss: 0.0814 - val_accuracy: 1.0000
Epoch 7/100
14/14 [==============================] - 0s 12ms/step - loss: 0.1029 - accuracy: 0.9909 - val_loss: 0.0846 - val_accuracy: 0.9909
Epoc

In [32]:
X_test_fwp = tf.convert_to_tensor(X_test_fwp, np.float32)
y_test_fwp = tf.convert_to_tensor(y_test_fwp, np.int16)
res_fwp = model_fwp.evaluate(X_test_fwp, y_test_fwp)

accuracy_dict['Falling Wedge class'] = res_fwp[1]

8/8 [==============================] - 0s 6ms/step - loss: 0.0822 - accuracy: 0.9958


# Double Top

In [33]:
X_real, y_real = real_data_pattern(patterns_to_test[2], model_type='categorise')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[2], model_type='categorise')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[2], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [34]:
y_all = np.array(y_joined)

X_train_dtp, X_test_dtp, y_train_dtp, y_test_dtp = train_test_split(X_pad, y_all, test_size=0.30)

X_train_dtp = tf.convert_to_tensor(X_train_dtp, np.float32)
y_train_dtp = tf.convert_to_tensor(y_train_dtp, np.int16)

In [35]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_dtp.shape[1:]
model_dtp = initialize_model_CNN(input_shape)

model_dtp.fit(
    X_train_dtp,
    y_train_dtp,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
14/14 [==============================] - 1s 22ms/step - loss: 10.6632 - accuracy: 0.6321 - val_loss: 2.2717 - val_accuracy: 0.7117
Epoch 2/100
14/14 [==============================] - 0s 14ms/step - loss: 3.3953 - accuracy: 0.6862 - val_loss: 1.5065 - val_accuracy: 0.5586
Epoch 3/100
14/14 [==============================] - 0s 14ms/step - loss: 1.0409 - accuracy: 0.7133 - val_loss: 0.4766 - val_accuracy: 0.7748
Epoch 4/100
14/14 [==============================] - 0s 13ms/step - loss: 0.4651 - accuracy: 0.8059 - val_loss: 0.4236 - val_accuracy: 0.8288
Epoch 5/100
14/14 [==============================] - 0s 12ms/step - loss: 0.4209 - accuracy: 0.8217 - val_loss: 0.7009 - val_accuracy: 0.8559
Epoch 6/100
14/14 [==============================] - 0s 12ms/step - loss: 0.3513 - accuracy: 0.8691 - val_loss: 0.3724 - val_accuracy: 0.8919
Epoch 7/100
14/14 [==============================] - 0s 12ms/step - loss: 0.8378 - accuracy: 0.8555 - val_loss: 1.4580 - val_accuracy: 0.7658
Epoch

In [36]:
X_test_dtp = tf.convert_to_tensor(X_test_dtp, np.float32)
y_test_dtp = tf.convert_to_tensor(y_test_dtp, np.int16)
res_dtp = model_dtp.evaluate(X_test_dtp, y_test_dtp)

accuracy_dict['Double Top class'] = res_dtp[1]

8/8 [==============================] - 0s 4ms/step - loss: 0.2255 - accuracy: 0.9412


# Double Bottom

In [37]:
X_real, y_real = real_data_pattern(patterns_to_test[3], model_type='categorise')
X_synthetic, y_synthetic = synthetic_data(X_real, patterns_to_test[3], model_type='categorise')
X_no_pattern, y_no_pattern = real_data_no_pattern(X_real, patterns_to_test[3], model_type='categorise')

X_joined = join_data(X_real, X_synthetic, X_no_pattern)
y_joined = join_data(y_real, y_synthetic, y_no_pattern)

X_pad = pad_arrays(X_joined)

In [38]:
y_all = np.array(y_joined)

X_train_dbp, X_test_dbp, y_train_dbp, y_test_dbp = train_test_split(X_pad, y_all, test_size=0.30)

X_train_dbp = tf.convert_to_tensor(X_train_dbp, np.float32)
y_train_dbp = tf.convert_to_tensor(y_train_dbp, np.int16)

In [39]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_dbp.shape[1:]
model_dbp = initialize_model_CNN(input_shape)

model_dbp.fit(
    X_train_dbp,
    y_train_dbp,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

Epoch 1/100
13/13 [==============================] - 1s 22ms/step - loss: 5.5328 - accuracy: 0.6398 - val_loss: 0.7445 - val_accuracy: 0.6800
Epoch 2/100
13/13 [==============================] - 0s 13ms/step - loss: 0.7100 - accuracy: 0.6146 - val_loss: 0.6352 - val_accuracy: 0.6600
Epoch 3/100
13/13 [==============================] - 0s 12ms/step - loss: 0.6415 - accuracy: 0.6146 - val_loss: 0.5371 - val_accuracy: 0.6900
Epoch 4/100
13/13 [==============================] - 0s 12ms/step - loss: 0.6023 - accuracy: 0.6625 - val_loss: 0.5045 - val_accuracy: 0.6900
Epoch 5/100
13/13 [==============================] - 0s 12ms/step - loss: 0.5615 - accuracy: 0.6599 - val_loss: 0.5038 - val_accuracy: 0.6900
Epoch 6/100
13/13 [==============================] - 0s 12ms/step - loss: 0.5218 - accuracy: 0.6549 - val_loss: 0.4329 - val_accuracy: 0.7300
Epoch 7/100
13/13 [==============================] - 0s 13ms/step - loss: 0.4901 - accuracy: 0.7053 - val_loss: 0.4135 - val_accuracy: 0.7300
Epoch 

In [40]:
X_test_dbp = tf.convert_to_tensor(X_test_dbp, np.float32)
y_test_dbp = tf.convert_to_tensor(y_test_dbp, np.int16)
res_dbp = model_dbp.evaluate(X_test_dbp, y_test_dbp)

accuracy_dict['Double Bottom class'] = res_dbp[1]

7/7 [==============================] - 0s 5ms/step - loss: 0.4130 - accuracy: 0.7617


# Results #2

In [41]:
accuracy_dict = pd.DataFrame(accuracy_dict, index=[0])
display("Accuracy DF")
display(accuracy_dict)
display("MAE DF")
display(results_df)

'Accuracy DF'

,Rising Wedge_class,Falling Wedge class,Double Top class,Double Bottom class
0,0.974138,0.995763,0.941176,0.761682


'MAE DF'

,Rising Wedge,Falling Wedge,Double Top,Double Bottom
1,22.038683,6.988884,25.853342,16.52389


# Model for determining dates

In [ ]:
X_real_rw, y_real_rw = real_data_pattern(patterns_to_test[0], model_type='full')
X_synthetic_rw, y_synthetic_rw = synthetic_data(X_real_rw, patterns_to_test[0], model_type='full', noise=False)

X_joined = join_data(X_real, X_synthetic)
y_joined = join_data(y_real, y_synthetic)

In [56]:
len(y_joined[50])

1

In [57]:
X_joined, y_joined = data_augmentation(X_joined, y_joined)

X_pad = pad_arrays(X_joined)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
y_all = np.array(y_joined)
X_train_rw, X_test_rw, y_train_rw, y_test_rw = train_test_split(X_pad, y_all, test_size=0.30)

X_train_rw = tf.convert_to_tensor(X_train_rw, np.float32)
y_train_rw = tf.convert_to_tensor(y_train_rw, np.int16)

In [ ]:
es = EarlyStopping(patience = 20, restore_best_weights=True)

input_shape = X_train_rw.shape[1:]
model_rw = initialize_model_CNN(input_shape)

model_rw.fit(
    X_train_rw,
    y_train_rw,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 100,
    callbacks = [es],
    verbose = 1
)

In [ ]:
X_test_rw = tf.convert_to_tensor(X_test_rw, np.float32)
y_test_rw = tf.convert_to_tensor(y_test_rw, np.int16)
res_rw = model_rw.evaluate(X_test_rw, y_test_rw)

results_dict['Rising Wedge'] = res_rw[1]